In [1]:
from transformers import (
    WhisperForConditionalGeneration,
    WhisperProcessor,
    WhisperConfig,
)
import torch
import ffmpeg
import torch
import torch.nn.functional as F
import numpy as np
from time import perf_counter
from pathlib import Path


In [2]:
# load_audio and pad_or_trim functions
SAMPLE_RATE = 16000
CHUNK_LENGTH = 30  # 30-second chunks
N_SAMPLES = CHUNK_LENGTH * SAMPLE_RATE  # 480000 samples in a 30-second chunk


In [3]:
# audio = whisper.load_audio('test.wav')
def load_audio(file: str, sr: int = SAMPLE_RATE, start_time: int = 0, dtype=np.float16):
    """
    Load an audio file into a numpy array at the specified sampling rate.
    """
    try:
        # This launches a subprocess to decode audio while down-mixing and resampling as necessary.
        # Requires the ffmpeg CLI and `ffmpeg-python` package to be installed.
        out, _ = (
            ffmpeg.input(file, ss=start_time, threads=0)
            .output("-", format="s16le", acodec="pcm_s16le", ac=1, ar=sr)
            .run(cmd=["ffmpeg", "-nostdin"], capture_stdout=True, capture_stderr=True)
        )
    except ffmpeg.Error as e:
        raise RuntimeError(f"Failed to load audio: {e.stderr.decode()}") from e

    # return np.frombuffer(out, np.int16).flatten().astype(np.float32) / 32768.0
    return np.frombuffer(out, np.int16).flatten().astype(dtype) / 32768.0


In [4]:
# audio = whisper.pad_or_trim(audio)
def pad_or_trim(array, length: int = N_SAMPLES, *, axis: int = -1):
    """
    Pad or trim the audio array to N_SAMPLES, as expected by the encoder.
    """
    if torch.is_tensor(array):
        if array.shape[axis] > length:
            array = array.index_select(
                dim=axis, index=torch.arange(length, device=array.device)
            )

        if array.shape[axis] < length:
            pad_widths = [(0, 0)] * array.ndim
            pad_widths[axis] = (0, length - array.shape[axis])
            array = F.pad(array, [pad for sizes in pad_widths[::-1] for pad in sizes])
    else:
        if array.shape[axis] > length:
            array = array.take(indices=range(length), axis=axis)

        if array.shape[axis] < length:
            pad_widths = [(0, 0)] * array.ndim
            pad_widths[axis] = (0, length - array.shape[axis])
            array = np.pad(array, pad_widths)

    return array


In [5]:
#TODO: Use relative path
_MODEL_PATH = '../../models/whisper/whisper-medium'

processor = WhisperProcessor.from_pretrained(_MODEL_PATH)
tokenizer = processor.tokenizer

config = WhisperConfig.from_pretrained(_MODEL_PATH)

model = WhisperForConditionalGeneration(config=config).from_pretrained(
    _MODEL_PATH,
    torch_dtype=torch.float16,
    # torch_dtype=torch.float32,
    low_cpu_mem_usage=True,
    # use_flash_attention_2=True
).to('cuda:0')
# model.to_bettertransformer()
model.eval()


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1024)
      (layers): ModuleList(
        (0-23): 24 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias

In [6]:
# model = torch.compile(model=model, backend='inductor')


In [7]:
_AUDIO_PATH = "../audio_samples/test/dev_wav/1_hi.wav"

tik = perf_counter()
audio = load_audio(_AUDIO_PATH)
audio = pad_or_trim(audio)

input_features = processor(audio, sampling_rate=SAMPLE_RATE, return_tensors="pt").input_features.to(model.device).to(dtype=model.dtype)
tok = perf_counter()

print(f'Inference time: {tok - tik:.2f}s')
from IPython.display import Audio
Audio(audio, rate=SAMPLE_RATE)


Inference time: 0.11s


In [8]:
input_features.shape


torch.Size([1, 80, 3000])

In [20]:
start_time = perf_counter()
with torch.no_grad():
    predicted_ids = model.generate(
        input_features,
        # num_beams=4,
        language='en',
        # task="transcribe",
        use_cache=True,
        # is_multilingual=False,
        return_timestamps=False,
    )
transcription = tokenizer.batch_decode(
    predicted_ids, 
    skip_special_tokens=True,
)[0]
end_time = perf_counter()

print(f'Inference time: {(end_time - start_time)*1000:.2f}ms')
print(transcription.strip())


Inference time: 468.07ms
Jarvis, what are you doing? I heard that you are selling new water.
